# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [134]:
!pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [135]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [136]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [137]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [138]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [166]:
text_splitter = CharacterTextSplitter(chunk_size=2000)
split_documents = text_splitter.split_texts(documents)
len(split_documents)

166

Let's take a look at some of the documents we've managed to split.

In [167]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [141]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

1. Yes, it can be reduced by setting the "dimensions" API parameter
2. Matryoshka Representation Learning (MRL)

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [168]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

Using async for embeddings to prevent blocking the application during the potentially slow I/O operation of fetching embeddings, thereby improving responsiveness and efficiency of the application. Using async calls is a common design pattern that helps improve the responsiveness of LLM-heavy applications.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [169]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ot the only time that it makes sense to micro66 The Pmarca Blog Archives\nmanage executives. It turns out that just about every executive\nin the world has a few things that are seriously wrong with\nthem. They have areas where they are truly deXcient in judgment or skill set. That’s just life. Almost nobody is brilliant\nat everything. When hiring and when Hring executives, you\nmust therefore focus on strength rather than lack of weakness. Everybody has severe weaknesses even if you can’t see\nthem yet. When managing, it’s oaen useful to micromanage and\nto provide remedial training around these weaknesses. Doing so\nmay make the diWerence between an executive succeeding or\nfailing.\nFor example, you might have a brilliant engineering executive\nwho generates excellent team loyalty, has terriXc product judgment and makes the trains run on time. This same executive\nmay be very poor at relating to the other functions in the company. She may generate far more than her share of cros

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

OpenAI recently introduced the *seed* input parameter, which can be used to produce "mostly" reproducible responses to a given prompt as long as temperature and top p are also kept the same. They also provide access to the *system_fingerprint* repsonse parameter which indicates model versioning on their side and helps make it a little more transparent when responses change due to system updates.

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [170]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [145]:
print(response)

The best way to write a loop in Python often depends on the specific task you want to accomplish, but there are some general guidelines to ensure your loops are clear, efficient, and easy to understand. Here are a few tips, along with examples for common loop types:

### 1. **Using `for` loops for finite iterations**

If you know the number of iterations in advance, the `for` loop is usually the most straightforward choice. It’s concise and readable.

```python
# Example: Print numbers 0 to 9
for i in range(10):
    print(i)
```

### 2. **Using `while` loops for indefinite iterations**

When the number of iterations is not known in advance, a `while` loop can be useful. Make sure to include a condition that eventually stops the loop to avoid infinite loops.

```python
# Example: Print numbers until a certain condition is met
count = 0
while count < 10:
    print(count)
    count += 1  # Don't forget to increment to avoid an infinite loop!
```

### 3. **List comprehensions for concise i

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [171]:
RAG_PROMPT_TEMPLATE = """ \
You are a seasoned startup coach that helps executives understand and mitigate the challenges facing their startup.
You should use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through the Week 1 Day 1 "Prompting OpenAI Like A Developer" material for an answer to this question!

There are many prompting techniques that could lead to a more detailed response, including chain-of-thought, instructing the system to take on a persona, including examples ('few-shot learning'), trying principle #6 from the "Principled Instructions" paper ('6 Add “I’m going to tip $xxx for a better solution!'), instructing the system to "think step by step", giving the system some guidelines on answer length and formatting, or others.

In [172]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [173]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")

{'response': "The 'Michael Eisner Memorial Weak Executive Problem' refers to the tendency of a CEO or startup founder to hire an executive who is weaker than themselves in a specific function, particularly one they were previously involved in. This often occurs because the CEO struggles to let go of the role they were once proficient in, leading them to select someone who is not as competent. The problem is exemplified by Michael Eisner, the former CEO of Disney, who after acquiring ABC, faced difficulties due to a poor performing network executive. The takeaway is to ensure that when hiring for a role that aligns with the founder's former specialty, the new hire should be significantly more capable than the founder in that area.",
 'context': [('ot the only time that it makes sense to micro66 The Pmarca Blog Archives\nmanage executives. It turns out that just about every executive\nin the world has a few things that are seriously wrong with\nthem. They have areas where they are truly 

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

There are many improvements that could be made to this pipeline! For example:
* Spell check the inputs, which appear to have many (intentional?) typos
* Context- (or at least word-) aware chunking
* A more nuanced and sophisticated system prompt
* Metadata support, for example including a date or blog entry title
* Adjust the OpenAI parameters to be more optimal for this use case

Recently, I have been working with some RAG pipelines that include hybrid search followed by cross encoders. Because cross encoders can be a bit heavyweight / slow depending on the approach, I have been wondering if a simple, more "classic" IR approach such as BM25 can work as a reranker. Although this remains a vector-only search, not hybrid, I decided to try this below.  I also made a couple other small tweaks:
* Increase the chunk size to 2k characters (about a page of text) to include a little more context
* Add a role to the system prompt

In [174]:
# Use BM25 to re-rank the retrieved results and see if it helps.
!pip install -qU rank_bm25 

from rank_bm25 import BM25Plus

# Utility function for reranking
def bm25plus_rerank(corpus, query, initial_ranking, top_n=3):
    tokenized_corpus = [corpus[i].split() for i in initial_ranking]
    tokenized_query = query.split()

    bm25 = BM25Plus(tokenized_corpus)
    bm25_scores = bm25.get_scores(tokenized_query)
    
    ranked_indices = [initial_ranking[i] for i in bm25_scores.argsort()[::-1]]    
    return ranked_indices[:top_n]

In [175]:

    
class RerankedQAPipeline(RetrievalAugmentedQAPipeline):
    # Extends the RetrievalAugmentedQAPipeline class with reranking
    
    def run_pipeline(self, user_query: str, rerank: bool=False, system_prompt: str = "") -> str:
        # Retrieve the top 10 results. Either return the top 3, or rerank with BM25 and then return 
        # the new top 3
        context_list = self.vector_db_retriever.search_by_text(user_query, k=10)

        # Convert from tuples to strings
        context_list_str = [context_list[i][0] for i in range(len(context_list))]

        # Optionally re-rank the retrieved context using BM25
        n = 3
        reranked_contexts = context_list_str[0:n]

        if rerank:
            initial_ranking = list(range(len(context_list_str)))
            reranked_indices = bm25plus_rerank(context_list_str, user_query, initial_ranking, top_n=n)
            reranked_contexts = [context_list_str[i] for i in reranked_indices]

        context_prompt = "\n\n".join(context for context in reranked_contexts) + "\n\n"

        formatted_system_prompt = SystemRolePrompt(system_prompt).create_message() if system_prompt else rag_prompt.create_message()
        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        # Step 5: Generate the response using the LLM
        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]),
            "context": reranked_contexts
        }

In [176]:
from pprint import pprint

# Utility function to make it easier to look at the contexts that are selected
def print_contexts(context_list: list[str]) -> None:
    print("**** Context ****")
    for i in range(0,len(context_list)):
            print("Context",i,":",context_list[i],"\n")
    print("\n")

# Utility function to compare two sets of responses
def compare_responses(pipeline:RerankedQAPipeline, query:str, show_context = False) -> None:
    no_reranking_response = retrieval_augmented_qa_pipeline.run_pipeline(query, rerank=False)
    reranked_response = retrieval_augmented_qa_pipeline.run_pipeline(query, rerank=True)

    pprint(f'Response without re-ranking: {no_reranking_response["response"]}')
    print("\n")
    pprint(f'Response with re-ranking: {reranked_response["response"]}')

    if show_context:
        print("\n\nBefore reranking:")
        print_contexts(no_reranking_response["context"])
        print("\n\nAfter reranking")
        print_contexts(reranked_response["context"])


In [180]:
retrieval_augmented_qa_pipeline = RerankedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

# Compare responses before and after re-ranking. The responses are pretty similar. 
query = "What is the 'Michael Eisner Memorial Weak Executive Problem'?"
compare_responses(retrieval_augmented_qa_pipeline,query, show_context=True)

("Response without re-ranking: The 'Michael Eisner Memorial Weak Executive "
 "Problem' refers to the tendency of CEOs or startup founders to hire "
 'executives for key functions who are weaker than themselves, particularly in '
 'the area that was their own specialty. This often happens because these '
 'leaders have difficulty letting go of the functions that initially brought '
 'them success. An example provided is Michael Eisner, the former CEO of '
 'Disney, who, despite being a brilliant TV network executive, encountered '
 'failure after acquiring ABC because he hired someone weak in that role, '
 'leading to poor performance. The cautionary tale highlights the importance '
 'of hiring individuals who are significantly more capable than the founder or '
 'CEO in their respective areas.')


('Response with re-ranking: The "Michael Eisner Memorial Weak Executive '
 'Problem" refers to the tendency of a CEO or startup founder to hire an '
 'executive who is weaker than themselves

In [181]:
# Try another query. Again, very similar responses
query = "What three risks that startups face?"
compare_responses(retrieval_augmented_qa_pipeline,query)

('Response without re-ranking: The three risks that startups face include:\n'
 '\n'
 '1. Founder risk — does the startup have the right founding team? This '
 'involves assessing whether the technologist is capable and whether the '
 'business person can effectively run the company.\n'
 '\n'
 '2. Market risk — is there a market for the product? This includes '
 'determining if anyone will want the product, if they will pay for it, and '
 'how much they will pay.\n'
 '\n'
 '3. Competition risk — are there too many other startups already doing this? '
 'This involves evaluating whether the startup is sufficiently differentiated '
 'from competitors and large incumbents.')


('Response with re-ranking: The three risks that startups face, as outlined in '
 'the provided context, include:\n'
 '\n'
 "1. Founder risk - This involves concerns about whether the founding team's "
 'skills and capabilities are sufficient for leading the startup, particularly '
 'the roles of the technologist and 

In [191]:
query = "What's the role of luck in entrepreneurship"
compare_responses(retrieval_augmented_qa_pipeline,query)

('Response without re-ranking: The role of luck in entrepreneurship is '
 'significant, as many successful entrepreneurs acknowledge that luck plays a '
 'huge part in the difference between success and failure. However, this '
 'acknowledgment is often made reluctantly because admitting the role of luck '
 'can undermine the image of the entrepreneur as a business genius.\n'
 '\n'
 'According to Dr. James Austin\'s theory outlined in his book "Chase, Chance, '
 'and Creativity," there are four kinds of luck:\n'
 '\n'
 '1. **Chance I**: Completely impersonal luck that cannot be influenced.\n'
 '2. **Chance II**: Luck that favors those who have a persistent curiosity and '
 'are willing to experiment.\n'
 '3. **Chance III**: Luck that benefits those with a strong background of '
 'knowledge and the ability to form new associations quickly.\n'
 '4. **Chance IV**: Luck that favors individuals with unique hobbies and '
 'lifestyles, allowing them to approach problems from unconventional an

Overall, it really isn't clear that re-ranking with BM25 leads to much improvement in responses, although I personally prefer the re-ranked response in the last example. Without a rigorous test dataset in place, it is hard to tell. Based on these results, I would probably consider other re-ranking methods first in the future.